# Profiling BatchFlow code

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

This notebooks shows how to profile various parts of BatchFlow: namely, pipelines and models.

In [1]:
import sys

sys.path.append("../../..")
from batchflow import B, V, W
from batchflow.opensets import MNIST
from batchflow.models.torch import ResNet18

In [2]:
dataset = MNIST()

To collect information about model training times (both on CPU and GPU), one must set `profile` option in the model configuration to `True`:

In [3]:
model_config = {
    'inputs/labels/classes': 10,
    'loss': 'ce',
    'profile': True,
}

In [4]:
pipeline = (dataset.train.p
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model('dynamic', ResNet18,
                            'resnet', config=model_config)
                .train_model('resnet',
                             B.images, B.labels,
                             fetches='loss',
                             save_to=V('loss_history', mode='a'))
                )

To gather statistics about how long each action takes, we must set `profile` to `True` inside `run` call:

In [5]:
BATCH_SIZE = 64
N_ITERS = 50

pipeline.run(BATCH_SIZE, n_iters=N_ITERS, bar=True, profile=True,
                   bar_desc=W(V('loss_history')[-1].format('Loss is {:7.7}')))

Loss is 0.1197789: 100%|██████████| 50/50 [00:57<00:00,  1.02s/it]


# Pipeline profiling

Now, pipeline has attribute `profile_info`: this `DataFrame` holds collected information:

In [6]:
pipeline.profile_info.head()

iter  total_time  \
action   id                                                                     
to_array <built-in method _abc._abc_instancecheck>::/hom...     1    0.108264   
         <built-in method _abc._abc_subclasscheck>::/hom...     1    0.108264   
         <built-in method _operator.index>::/home/tsimfe...     1    0.108264   
         <built-in method posix.sched_getaffinity>::../....     1    0.108264   
         <built-in method builtins.any>::../../../batchf...     1    0.108264   

                                                             pipeline_time  \
action   id                                                                  
to_array <built-in method _abc._abc_instancecheck>::/hom...       0.101738   
         <built-in method _abc._abc_subclasscheck>::/hom...       0.101738   
         <built-in method _operator.index>::/home/tsimfe...       0.101738   
         <built-in method posix.sched_getaffinity>::../....       0.101738   
         <built-in method builtins.any>::../../../batchf...       0.101738   

                                                             ncalls   tottime  \
action   id                                                                     
to_array <built-in method _abc._abc_instancecheck>::/hom...      64  0.000214   
         <built-in method _abc._abc_subclasscheck>::/hom...      57  0.000730   
         <built-in method _operator.index>::/home/tsimfe...     128  0.000067   
         <built-in method posix.sched_getaffinity>::../....       1  0.000014   
         <built-in method builtins.any>::../../../batchf...       1  0.000008   

                                                              cumtime  \
action   id                                                             
to_array <built-in method _abc._abc_instancecheck>::/hom...  0.001107   
         <built-in method _abc._abc_subclasscheck>::/hom...  0.000887   
         <built-in method _operator.index>::/home/tsimfe...  0.000067   
         <built-in method posix.sched_getaffinity>::../....  0.000014   
         <built-in method builtins.any>::../../../batchf...  0.000031   

                                                                    batch_id  \
action   id                                                                    
to_array <built-in method _abc._abc_instancecheck>::/hom...  139619470154496   
         <built-in method _abc._abc_subclasscheck>::/hom...  139619470154496   
         <built-in method _operator.index>::/home/tsimfe...  139619470154496   
         <built-in method posix.sched_getaffinity>::../....  139619470154496   
         <built-in method builtins.any>::../../../batchf...  139619470154496   

                                                               start_time  
action   id                                                                
to_array <built-in method _abc._abc_instancecheck>::/hom...  1.582117e+09  
         <built-in method _abc._abc_subclasscheck>::/hom...  1.582117e+09  
         <built-in method _operator.index>::/home/tsimfe...  1.582117e+09  
         <built-in method posix.sched_getaffinity>::../....  1.582117e+09  
         <built-in method builtins.any>::../../../batchf...  1.582117e+09

Note that there is a detailed information about exact methods that are called inside each of the actions. That is a lot of data which can give us precise understanding of parts of the code, that are our bottlenecks.

Columns of the `profile_info`:
- `action`, `iter`, `batch_id` and `start_time` are pretty self-explainable
- `id` allows to identify exact method with great details: it is a concatenation of `method_name`, `file_name`, `line_number` and `callee`
- `total_time` is a time taken by an action
- `pipeline_time` is `total_time` plus time of processing the profiling table at each iteration
- `tottime` is a time taken by a method inside action
- `cumtime` is a time take by a method and all of the methods that are called inside this method


More often than not, though, we don't need such granularity. Pipeline method `show_profile_info` makes some handy aggregations:

**Note:** by default, results are sorted on `total_time` or `tottime`, depending on level of details.

In [7]:
# timings for each action
pipeline.show_profile_info(per_iter=False, detailed=False)

total_time                     pipeline_time                    
                   sum      mean       max           sum      mean       max
action                                                                      
train_model  51.890390  1.037808  1.643089     51.642231  1.032845  1.635717
to_array      2.209763  0.044195  0.109565      1.967972  0.039359  0.101738
multiply      1.330352  0.026607  0.052029      1.167702  0.023354  0.049151

In [8]:
# for each action show 2 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=False, detailed=True, sortby=('ncalls', 'max'), limit=2)

ncalls  \
                                                                    sum   
action      id                                                            
multiply    <built-in method builtins.isinstance>::../../.....   409600   
            <built-in method numpy.where>::../../../batchfl...   204800   
to_array    <built-in method builtins.isinstance>::../../.....   409600   
            <built-in method numpy.where>::../../../batchfl...   204800   
train_model <method 'append' of 'list' objects>::/home/tsim...  1082936   
            <lambda>::~::0::<method 'sort' of 'list' objects>    541468   

                                                                          \
                                                                    mean   
action      id                                                             
multiply    <built-in method builtins.isinstance>::../../.....   8192.00   
            <built-in method numpy.where>::../../../batchfl...   4096.00   
to_array    <built-in method builtins.isinstance>::../../.....   8192.00   
            <built-in method numpy.where>::../../../batchfl...   4096.00   
train_model <method 'append' of 'list' objects>::/home/tsim...  21658.72   
            <lambda>::~::0::<method 'sort' of 'list' objects>   10829.36   

                                                                       \
                                                                  max   
action      id                                                          
multiply    <built-in method builtins.isinstance>::../../.....   8192   
            <built-in method numpy.where>::../../../batchfl...   4096   
to_array    <built-in method builtins.isinstance>::../../.....   8192   
            <built-in method numpy.where>::../../../batchfl...   4096   
train_model <method 'append' of 'list' objects>::/home/tsim...  21686   
            <lambda>::~::0::<method 'sort' of 'list' objects>   10843   

                                                                 tottime  \
                                                                     sum   
action      id                                                             
multiply    <built-in method builtins.isinstance>::../../.....  0.040543   
            <built-in method numpy.where>::../../../batchfl...  0.094183   
to_array    <built-in method builtins.isinstance>::../../.....  0.058232   
            <built-in method numpy.where>::../../../batchfl...  0.138339   
train_model <method 'append' of 'list' objects>::/home/tsim...  0.101479   
            <lambda>::~::0::<method 'sort' of 'list' objects>   0.068769   

                                                                          \
                                                                    mean   
action      id                                                             
multiply    <built-in method builtins.isinstance>::../../.....  0.000811   
            <built-in method numpy.where>::../../../batchfl...  0.001884   
to_array    <built-in method builtins.isinstance>::../../.....  0.001165   
            <built-in method numpy.where>::../../../batchfl...  0.002767   
train_model <method 'append' of 'list' objects>::/home/tsim...  0.002030   
            <lambda>::~::0::<method 'sort' of 'list' objects>   0.001375   

                                                                          \
                                                                     max   
action      id                                                             
multiply    <built-in method builtins.isinstance>::../../.....  0.001351   
            <built-in method numpy.where>::../../../batchfl...  0.003194   
to_array    <built-in method builtins.isinstance>::../../.....  0.003015   
            <built-in method numpy.where>::../../../batchfl...  0.008334   
train_model <method 'append' of 'list' objects>::/home/tsim...  0.004265   
            <lambda>::~::0::<method 'sort' of 'list' objects>   0.00

In [9]:
# timings for each action for each iter
pipeline.show_profile_info(per_iter=True, detailed=False,)

total_time  pipeline_time         batch_id
iter action                                                 
1    train_model    1.643089       1.635717  139619470154496
     to_array       0.108264       0.101738  139619470154496
     multiply       0.021203       0.018426  139619470154496
2    train_model    1.248317       1.231082  139619470155168
     to_array       0.033515       0.027055  139619470155168
     multiply       0.029224       0.023848  139619470155168
3    train_model    0.945223       0.940927  139619456921048
     to_array       0.059107       0.055046  139619456921048
     multiply       0.030329       0.027060  139619456921048
4    train_model    1.127812       1.123748  139619470246688
     to_array       0.046803       0.042963  139619470246688
     multiply       0.020962       0.018041  139619470246688
5    train_model    1.056066       1.048639  139619470248480
     multiply       0.047510       0.044703  139619470248480
     to_array       0.039407       0.033004  139619470248480
6    train_model    1.163227       1.159149  139619887129712
     to_array       0.042080       0.039059  139619887129712
     multiply       0.025147       0.022363  139619887129712
7    train_model    1.030484       1.025648  139619395546192
     to_array       0.036669       0.033544  139619395546192
     multiply       0.031736       0.028950  139619395546192
8    train_model    0.957532       0.948904  139619470246688
     to_array       0.035956       0.028389  139619470246688
     multiply       0.028239       0.025259  139619470246688
9    train_model    0.967270       0.962909  139619456920656
     to_array       0.042441       0.037243  139619456920656
     multiply       0.024061       0.020818  139619456920656
10   train_model    1.109577       1.105219  139619470248480
     to_array       0.036515       0.033258  139619470248480
     multiply       0.020640       0.017751  139619470248480
...                      ...            ...              ...
41   train_model    1.453717       1.433182  139619246945056
     to_array       0.031609       0.027132  139619246945056
     multiply       0.020542       0.017809  139619246945056
42   train_model    1.194791       1.190733  139619395457600
     to_array       0.109565       0.098708  139619395457600
     multiply       0.029016       0.026182  139619395457600
43   train_model    0.903590       0.899502  139619373805920
     to_array       0.034272       0.029177  139619373805920
     multiply       0.022923       0.020129  139619373805920
44   train_model    0.955875       0.951854  139619243745520
     to_array       0.035425       0.031957  139619243745520
     multiply       0.023273       0.020472  139619243745520
45   train_model    0.908158       0.904168  139619246945056
     to_array       0.036164       0.027513  139619246945056
     multiply       0.026482       0.021973  139619246945056
46   train_model    0.959161       0.955147  139619395459784
     to_array       0.052879       0.049157  139619395459784
     multiply       0.026890       0.022510  139619395459784
47   train_model    0.984970       0.980959  139619373805920
     to_array       0.032078       0.024607  139619373805920
     multiply       0.023910       0.021025  139619373805920
48   train_model    0.877188       0.873139  139619243745520
     to_array       0.037004       0.031564  139619243745520
     multiply       0.021250       0.018427  139619243745520
49   train_model    0.847959       0.843947  139619232417160
     to_array       0.038785       0.035736  139619232417160
     multiply       0.027238       0.024441  139619232417160
50   train_model    0.871393       0.867344  139619243726944
     to_array       0.060779       0.055249  139619243726944
     multiply       0.020711       0.017943  139619243726944

[150 rows x 3 columns]

In [10]:
# for each iter each action show 3 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=True, detailed=True, sortby='tottime', limit=3)

ncalls  \
iter action      id                                                           
1    multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     to_array    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     train_model <built-in method conv2d>::/home/tsimfer/anacond...     143   
                 <method 'run_backward' of 'torch._C._EngineBase...       1   
                 <built-in method batch_norm>::/home/tsimfer/ana...     143   
2    multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 _get::../../../batchflow/batchflow/components.p...      64   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
     to_array    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     train_model <method 'run_backward' of 'torch._C._EngineBase...       1   
                 __init__::/home/tsimfer/anaconda3/lib/python3.7...   10843   
                 parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
3    multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 _get::../../../batchflow/batchflow/components.p...      64   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
     to_array    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     train_model <method 'run_backward' of 'torch._C._EngineBase...       1   
                 <built-in method conv2d>::/home/tsimfer/anacond...      20   
                 parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
4    multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
...                                                                     ...   
47   train_model <method 'run_backward' of 'torch._C._EngineBase...       1   
                 <built-in method conv2d>::/home/tsimfer/anacond...      20   
                 parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
48   multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     to_array    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     train_model <method 'run_backward' of 'torch._C._EngineBase...       1   
                 <built-in method conv2d>::/home/tsimfer/anacond...      20   
                 parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
49   multiply    find_in_index::../../../batchflow/batchflow/com...    4096   
                 _get::../../../batchflow/batchflow/components.p...      64   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
     to_array    find_in_index::../../../batchflow/batchflow/com...    4096   
                 <listcomp>::../../../batchflow/batchflow/compon...      64   
                 <built-in method numpy.where>::../../../batchfl...    4096   
     train_model <method 'run_backward' 

# Model profiling

In [11]:
model = pipeline.m('resnet')

There is an `info` property that, unsurprisingly, shows a lot of interesting details regarding model itself or the training process:

In [12]:
model.info


##### Config:
{'benchmark': True,
 'body': {'encoder': {'blocks': {'base': <class 'batchflow.models.torch.blocks.ResBlock'>,
                                 'bottleneck': False,
                                 'downsample': [False, True, True, True],
                                 'filters': [64, 128, 256, 512],
                                 'layout': 'cnacn',
                                 'n_reps': [2, 2, 2, 2],
                                 'se': False},
                      'downsample': {'layout': 'p',
                                     'pool_size': 2,
                                     'pool_strides': 2},
                      'num_stages': 4,
                      'order': ['skip', 'block']}},
 'common': {'data_format': 'channels_first'},
 'decay': None,
 'device': None,
 'head': {'classes': 10,
          'dropout_rate': 0.4,
          'filters': 10,
          'layout': 'Vdf',
          'target_shape': (64,),
          'units': 10},
 'initial_block': {'filters'

As with pipeline, there is a `profile_info` attribute, as well as `show_profile_info` method. Depending on type of the used device (`CPU` or `GPU`)

In [13]:
# one row for every operation inside model; limit at 5 rows
model.show_profile_info(per_iter=False, limit=5)

ncalls              CPU_tottime            \
                               sum    mean  max         sum      mean   
name                                                                    
mkldnn_convolution_backward   1000   20.00   20   25.109348  0.502187   
mkldnn_convolution            1000   20.00   20    4.376594  0.087532   
sqrt                          3038   62.00   62    2.756476  0.056255   
native_batch_norm_backward    1000   20.00   20    1.811546  0.036231   
add_                         10114  202.28  206    1.408720  0.028174   

                                      CPU_cumtime                      \
                                  max         sum      mean       max   
name                                                                    
mkldnn_convolution_backward  0.750240   25.109348  0.502187  0.750240   
mkldnn_convolution           0.143996    4.376594  0.087532  0.143996   
sqrt                         0.079583    2.756476  0.056255  0.079583   
native_batch_norm_backward   0.067658    1.811546  0.036231  0.067658   
add_                         0.047941    1.408720  0.028174  0.047941   

                            CPU_tottime_avg                      
                                        sum      mean       max  
name                                                             
mkldnn_convolution_backward        1.255467  0.025109  0.037512  
mkldnn_convolution                 0.218830  0.004377  0.007200  
sqrt                               0.044459  0.000907  0.001284  
native_batch_norm_backward         0.090577  0.001812  0.003383  
add_                               0.006846  0.000137  0.000233

In [14]:
# for each iteration show 3 of the slowest operations
model.show_profile_info(per_iter=True, limit=3)

ncalls  CPU_tottime  CPU_cumtime  \
iter name                                                            
0    mkldnn_convolution_backward      20     0.486976     0.486976   
     mkldnn_convolution               20     0.091332     0.091332   
     native_batch_norm_backward       20     0.039047     0.039047   
1    mkldnn_convolution_backward      20     0.450588     0.450588   
     mkldnn_convolution               20     0.084945     0.084945   
     sqrt                             62     0.079583     0.079583   
2    mkldnn_convolution_backward      20     0.481878     0.481878   
     mkldnn_convolution               20     0.081743     0.081743   
     sqrt                             62     0.061165     0.061165   
3    mkldnn_convolution_backward      20     0.497236     0.497236   
     mkldnn_convolution               20     0.077387     0.077387   
     sqrt                             62     0.066028     0.066028   
4    mkldnn_convolution_backward      20     0.562705     0.562705   
     mkldnn_convolution               20     0.076460     0.076460   
     native_batch_norm_backward       20     0.047757     0.047757   
5    mkldnn_convolution_backward      20     0.508858     0.508858   
     mkldnn_convolution               20     0.107517     0.107517   
     sqrt                             62     0.062212     0.062212   
6    mkldnn_convolution_backward      20     0.466723     0.466723   
     mkldnn_convolution               20     0.102556     0.102556   
     sqrt                             62     0.066445     0.066445   
7    mkldnn_convolution_backward      20     0.476079     0.476079   
     mkldnn_convolution               20     0.080959     0.080959   
     sqrt                             62     0.065917     0.065917   
8    mkldnn_convolution_backward      20     0.490666     0.490666   
     mkldnn_convolution               20     0.077444     0.077444   
     sqrt                             62     0.061876     0.061876   
9    mkldnn_convolution_backward      20     0.457111     0.457111   
     mkldnn_convolution               20     0.077742     0.077742   
     sqrt                             62     0.069420     0.069420   
...                                  ...          ...          ...   
40   mkldnn_convolution_backward      20     0.530369     0.530369   
     mkldnn_convolution               20     0.079005     0.079005   
     sqrt                             62     0.064134     0.064134   
41   mkldnn_convolution_backward      20     0.632706     0.632706   
     mkldnn_convolution               20     0.109878     0.109878   
     sqrt                             62     0.063744     0.063744   
42   mkldnn_convolution_backward      20     0.437805     0.437805   
     mkldnn_convolution               20     0.081218     0.081218   
     sqrt                             62     0.064935     0.064935   
43   mkldnn_convolution_backward      20     0.480136     0.480136   
     mkldnn_convolution               20     0.077706     0.077706   
     sqrt                             62     0.063577     0.063577   
44   mkldnn_convolution_backward      20     0.443208     0.443208   
     mkldnn_convolution               20     0.076679     0.076679   
     sqrt                             62     0.066144     0.066144   
45   mkldnn_convolution_backward      20     0.456552     0.456552   
     mkldnn_convolution               20     0.095810     0.095810   
     sqrt                             62     0.061868     0.061868   
46   mkldnn_convolution_backward      20     0.475134     0.475134   
     mkldnn_convolution               20     0.097313     0.097313   
     sqrt                             62     0.067705     0.067705   
47   mkldnn_convolution_backward      20     0.468701     0.468701   
     mkldnn_convolution               20     0.076955     0.076955   
     sqrt                             62     0.037484     0.037484   
48   mkldnn_convolution_backward      20    